StyleGAN3

In [58]:
#@markdown ###Check GPU type
#@markdown Factory reset runtime if you don't have the desired GPU.
#@markdown - V100 = Excellent (*Available only for Colab Pro users*)
#@markdown - P100 = Very Good
#@markdown - T4 = Good (*preferred*)
#@markdown - K80 = Meh
#@markdown - P4 = (*Not Recommended*)

!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-6aebdffa-906a-0659-7cc2-1f760f176747)


In [59]:
NAME = 'stylegan3'
GDRIVE_MOUNT = '/content/drive'
BASE_DIR = f'{GDRIVE_MOUNT}/MyDrive/colab/'
CODE_DIR = f'{BASE_DIR}{NAME}/'

In [62]:
# Connect Google Drive
from google.colab import drive
drive.mount(GDRIVE_MOUNT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
import os
if os.path.isdir(CODE_DIR):
    %cd {CODE_DIR}
    !git pull
else:
    !git clone https://github.com/ManuelKugelmann/stylegan3-fun.git {CODE_DIR}
    %cd {CODE_DIR}

/content/drive/MyDrive/colab/stylegan3
Already up to date.
